# Simple example of matching based on proximity
Matching two catalogs based on proximity based on a configuration dictionary

In [ ]:
%load_ext autoreload
%autoreload 2

## Catalogs
Given some input data

In [ ]:
import numpy as np
from astropy.table import Table
input1 = Table({ 
    'ID': [f'CL{i}' for i in range(5)],
    'RA': [0.0, 0.0001, 0.00011, 25, 20],
    'DEC': [0.0, 0.0, 0.0, 0.0, 0.0],
    'Z': [0.2, 0.3, 0.25, 0.4, 0.35],
    'MASS': [10**13.5, 10**13.4, 10**13.3, 10**13.8, 10**14],
    'RADIUS_ARCMIN': [1.0, 1.0, 1.0, 1.0, 1.0],
})
input2 = Table({
    'ID': ['CL0', 'CL1', 'CL2', 'CL3'],
    'RA': [0.0, 0.0001, 0.00011, 25],
    'DEC': [0.0, 0, 0, 0],
    'Z': [0.3, 0.2, 0.25, 0.4],
    'MASS': [10**13.3, 10**13.4, 10**13.5, 10**13.8],
    'RADIUS_ARCMIN': [1.0, 1.0, 1.0, 1.0],
})

In [ ]:
display(input1)
display(input2)

Create two `Catalog` objects:

In [ ]:
from clevar.catalog import Catalog
c1 = Catalog('Cat1', id=input1['ID'], ra=input1['RA'], dec=input1['DEC'], z=input1['Z'], mass=input1['MASS'])
c2 = Catalog('Cat2', id=input2['ID'], ra=input2['RA'], dec=input2['DEC'], z=input2['Z'], mass=input2['MASS'])
display(c1)
display(c2)

## Matching
Import the `ProximityMatch` and create a object for matching

In [ ]:
from clevar.match.proximity import ProximityMatch
mt = ProximityMatch()

Prepare the configuration. The main values are:

- `type`: Type of matching to be considered. Can be a simple match of Catalog1->Catalog2 (`cat1`), Catalog2->Catalog1 (`cat2`) or cross matching.
- `which_radius`: Given a pair of clusters, which radius will be used for the matching.
- `preference`: In cases where there are multiple matched, how the best candidate will be chosen.

We also need to provide some specific configuration for each catalog with:

- `delta_z`: Defines redshift window for matching. The possible values are:
  - `'cat'`: uses redshift properties of the catalog
  - `'spline.filename'`: interpolates data in `'filename'` assuming (z, zmin, zmax) format
  - `float`: uses `delta_z*(1+z)`
  - `None`: does not use z
- `match_radius`: Radius of the catalog to be used in the matching. If `'cat'` uses the radius in the catalog, else must be in format `'value unit'`. (ex: `'1 arcsec'`, `'1 Mpc'`)

In this case, because one of the configuraion radius has physical units, we need a cosmology object to convert it to angular size (this is done internally).

In [ ]:
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'max', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'angular_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':.2,
                'match_radius': '1 mpc'
                },
    'catalog2': {'delta_z':.2,
                'match_radius': '10 arcsec'
                }
}
from clevar.cosmology import AstroPyCosmology
cosmo = AstroPyCosmology()

Once the configuration is prepared, the whole process can be done with one call:

In [ ]:
%%time
mt.match_from_config(c1, c2, match_config, cosmo=cosmo)

This will fill the matching columns in the catalogs:
- `mt_multi_self`: Multiple matches found
- `mt_multi_other`: Multiple matches found by the other catalog
- `mt_self`: Best candidate found
- `mt_other`: Best candidate found by the other catalog
- `mt_cross`: Best candidate found in both directions

In [ ]:
display(c1)
display(c2)

# Save and Load
The results of the matching can easily be saved and load using `ClEvaR` tools:

In [ ]:
mt.save_matches(c1, c2, out_dir='temp', overwrite=True)

In [ ]:
mt.load_matches(c1, c2, out_dir='temp')
display(c1)
display(c2)